# Mapping Pixels to Subpixels and creating a dict

### 1. Map every pixel to 2k subpixels

- Get center of subpixels using pix2ang
- Then use theta, phi = raDec2thetaPhi(ra, dec) for all subpixels
- pixel_indices = hp.ang2pix(NSIDE, theta,phi)
- Maps the centers of all subpixels to the 256 pixels

In [1]:
import numpy as np
from astropy.io import fits
import healpy as hp
import pandas as pd
import matplotlib.path as mplPath
import matplotlib.pyplot as plt
import pickle

In [ ]:
def raDec2thetaPhi(ra, dec):
    return (0.5 * np.pi - np.deg2rad(dec)), (np.deg2rad(ra))

scaling_factor = 8

NSIDE = 256
NPIX = hp.nside2npix(NSIDE)
NSIDE_SUB = NSIDE * scaling_factor
NPIX_SUB = hp.nside2npix(NSIDE_SUB)
print(NSIDE_SUB, NPIX_SUB)

subpixels = np.arange(0,NPIX_SUB)
print(len(subpixels))

In [ ]:
# Getting centers of subpixels and mapping subpixels 2 pixels

subpixel_coords = hp.pix2ang(NSIDE_SUB,subpixels)
theta_sub= subpixel_coords[0]
phi_sub = subpixel_coords[1]
pixel_indices = hp.ang2pix(NSIDE, theta_sub,phi_sub)
print(len(pixel_indices))

In [ ]:
# Verifying that pixelation worked accordingly

unique = np.unique(pixel_indices)
print(len(unique))

In [ ]:
pixel2subpixel_dict = {}

#Loop through the assigned pixel of every subpixel and fill a dict
for subpixel, pixel in enumerate(pixel_indices):
    if pixel_indices[subpixel] not in pixel2subpixel_dict:
        pixel2subpixel_dict[pixel] = []
    pixel2subpixel_dict[pixel].append(subpixel)

In [ ]:
# Verify previous step worked by counting list lenght per pixel

#Every Pixel features: This must equal the len of unique pixel NPIX
pixel_count = len(pixel2subpixel_dict.keys())
print(pixel_count)

#Verifying that the 64 subpixels are approximately associated to each pixel
subpixel_count = 0
min = 200
max = 0
for elem in pixel2subpixel_dict.keys():
    elems_in_pixel = len(pixel2subpixel_dict[elem])
    if elems_in_pixel < min:
        min = elems_in_pixel
    if elems_in_pixel > max:
        max = elems_in_pixel
    subpixel_count += elems_in_pixel

print(min)
print(max)
print(subpixel_count)
print(subpixel_count/pixel_count)


In [ ]:
# THIS CODE IS NEEDED WHEN A NEW PIXEL2SUBPIXEL DICT WAS CREATED

''' 
with open('../bricks_data/pixel2subpixel_256_2048.pickle', 'wb') as f:
    pickle.dump(pixel2subpixel_dict, f)
    f.close()
'''

In [ ]:
#Plotting Pixel 2 SubPixel Mapping

subpixels = pixel2subpixel_dict[459294]


#Plot Subpixels
for subpix in subpixels:
    pixel_boundary = hp.boundaries(nside=NSIDE_SUB, pix=subpix, step=1)
    pixel_boundary_raDec = hp.vec2ang(pixel_boundary.transpose(),lonlat=True)
    pixel_boundary_raDec = list(pixel_boundary_raDec)
    pixel_boundary_raDec[0] = np.append(pixel_boundary_raDec[0],pixel_boundary_raDec[0][0])
    pixel_boundary_raDec[1] = np.append(pixel_boundary_raDec[1],pixel_boundary_raDec[1][0])
    plt.plot(pixel_boundary_raDec[0],pixel_boundary_raDec[1], c='red')

#Plot Larger Pixel
pixel_boundary = hp.boundaries(nside=NSIDE, pix=459294, step=1)
pixel_boundary_raDec = hp.vec2ang(pixel_boundary.transpose(),lonlat=True)
pixel_boundary_raDec = list(pixel_boundary_raDec)
pixel_boundary_raDec[0] = np.append(pixel_boundary_raDec[0],pixel_boundary_raDec[0][0])
pixel_boundary_raDec[1] = np.append(pixel_boundary_raDec[1],pixel_boundary_raDec[1][0])
plt.plot(pixel_boundary_raDec[0],pixel_boundary_raDec[1], c='black', label="Pixel Boundary")


plt.legend()
plt.show()
